In [163]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import itertools

segment = 'method' # 'contract' # 'project'

In [183]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df[df['project'] != '0x4200000000000000000000000000000000000006']
df = df[df['project'] != '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2']
df = df.fillna(0)

addr_list = df[['address','chain']].drop_duplicates()
addr_list.to_csv('csv_outputs/whale_list.csv')


In [165]:
df_merge = df.copy()
df_merge['usd_value_in'] = np.where(df_merge['token_direction'] == 'in',df_merge['usd_value'], 0).astype(np.float64)
df_merge['usd_value_out'] = np.where(df_merge['token_direction'] == 'out',df_merge['usd_value'], 0).astype(np.float64)
df_merge = df_merge.groupby(['chain','address','project','contract','method']).sum()
df_merge = df_merge.reset_index()

df_merge['usd_value_net'] =  df_merge['usd_value_out'] - df_merge['usd_value_in'] # net in contracts
df_merge['usd_value_net'] = np.where(df_merge['usd_value_net'] < 0, 0, df_merge['usd_value_net'])
# df_merge['total_addrs'] = df_merge.groupby(['chain'])['address'].transform(lambda x: x.nunique())

val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

display(df_merge.sample(20))

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/2435008623.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_merge = df_merge.groupby(['chain','address','project','contract','method']).sum()


,chain,address,project,contract,method,usd_value,num_tfers,num_txs,usd_value_in,usd_value_out,usd_value_net
106318,Ethereum,0x6c037c365536c5257bea20f46be4f2e16f7dc2a9,0xa8af5c003a5028eda5de0d3de6b1ecfe35445f0f,0,transfer,2.815288e+05,1,1,2.815288e+05,0.000000e+00,0.000000e+00
242014,Ethereum,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,0xc6ad3cdf68f5ce7fa93e3e8cbe272f47898de740,0,transfer,2.059724e+04,1,1,2.059724e+04,0.000000e+00,0.000000e+00
34392,Ethereum,0x1f8e9119872bc0bb830edb312884fb6464bd605a,0xf9b8551d40f4d347960d41b065cfe71584f235a9,0,transfer,3.021145e+05,1,1,0.000000e+00,3.021145e+05,3.021145e+05
207869,Ethereum,0xd1ae97f26303639a17f8719971bab61e6f851b9e,0x5b7e96a27cf00a60af4ded4caed29ed00b1e0bae,0,transfer,2.014164e+04,1,1,0.000000e+00,2.014164e+04,2.014164e+04
136105,Ethereum,0x87a3bdf4addc99b4fb2285d08b35d7589cfda0cf,OTHER,OTHER,OTHER,1.050195e+02,5,5,1.050195e+02,0.000000e+00,0.000000e+00
108174,Ethereum,0x6dfc34609a05bc22319fa4cce1d1e2929548c0d7,0x1ae94b1376c6e0000e6dfe515714a5170e06badc,0,transfer,6.010651e+04,1,1,0.000000e+00,6.010651e+04,6.010651e+04
133045,Ethereum,0x8397e27f2118905ee7c356dbd3bcf399eb110f88,0xac13d2a9bb3bc46ae55bdac4e2bc862c0e96f488,0,transfer,3.013032e+04,1,1,3.013032e+04,0.000000e+00,0.000000e+00
21250,Ethereum,0x0ea2ebccccfe42a42548845b983905a3ee7d70f2,OTHER,OTHER,OTHER,4.181436e+03,2,2,4.181436e+03,0.000000e+00,0.000000e+00
3270,Arbitrum,0x8d5d3906d2bddade3c867b7d9ff02e901125895a,OTHER,OTHER,OTHER,8.182084e+03,49,49,5.790771e+03,2.391313e+03,0.000000e+00
49115,Ethereum,0x3018018c44338b9728d02be12d632c6691e020d1,0x3926b2ab78d05561e586d7a4c1ad0e037893040f,0,transferFrom,1.895736e+04,2,2,1.895736e+04,0.000000e+00,0.000000e+00


In [166]:
addr_cols = ['chain','address','project','contract','method']
df_addr = df_merge.groupby(addr_cols).sum()
df_addr.reset_index(inplace=True)
# Create a list of all possible combinations
df_addr = df_addr.pivot_table(index=['chain','address','project','contract','method'], \
                                     values=val_cols, fill_value=0)
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
display(df_addr.sample(20))

,chain,address,project,contract,method,usd_value,usd_value_in,usd_value_net,usd_value_out,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share
112277,Ethereum,0x71671766249febf252e42b5a750646f6694cf7fd,0x10df5fe7236e1fa94eabb6c3e263d58cc01c78e6,0,transfer,2.257670e+04,2.257670e+04,0.000000e+00,0.000000e+00,0.002106,0.003564,0.000000,0.000000
41338,Ethereum,0x2815b1439ce47acfda42db7d89044e7d2629bee8,0x0e9cf44eb29d82434aa16913e2f939b7b73f3fb9,0,transfer,4.427334e+04,0.000000e+00,4.427334e+04,4.427334e+04,0.000183,0.000000,0.000366,0.000367
225714,Ethereum,0xe6a3ae442bcbe7b5ece5858bda43a170e5374cfb,0xe3557d57f46b5e2ac382fa9d374f1ee0641f1a66,0,transfer,1.580303e+05,0.000000e+00,1.580303e+05,1.580303e+05,0.001312,0.000000,0.002624,0.002638
205750,Ethereum,0xcf3abce173aab9ba2047255397d83da4c372b765,0xcf63e99c9b6150508d0d8e05ccee764dde980d29,0,transfer,5.967893e+04,0.000000e+00,5.967893e+04,5.967893e+04,0.017348,0.000000,0.034156,0.039812
72832,Ethereum,0x4849d6ad05a8937c00cd8e3f37a294c289b5afe2,0x5908e4f4128d74d5c8eccb78429cc6ab1f10e5cd,0,transfer,3.977974e+04,3.977974e+04,0.000000e+00,0.000000e+00,0.000260,0.000520,0.000000,0.000000
197008,Ethereum,0xc7c90734f3ee63e477d2ead1d02f3ebaa9140a58,0x20940b3db5896fca33ca878b4e9ebe3cb0a17761,0,transfer,2.510890e+04,2.510890e+04,0.000000e+00,0.000000e+00,0.021798,0.043596,0.000000,0.000000
100633,Ethereum,0x653c2b0343e10eb29726663094ca5c41d514c7af,0x1d129a6f8197f9c29e637eca81e7daec781f2443,0,transfer,2.486585e+06,0.000000e+00,2.486585e+06,2.486585e+06,0.999906,NaN,0.999906,0.999906
125038,Ethereum,0x7c9c7acef3273f696cc6eeab621e7429e349d7d6,OTHER,OTHER,OTHER,2.771205e+04,1.648185e+04,0.000000e+00,1.123019e+04,0.002898,0.003724,0.002186,0.000000
126262,Ethereum,0x7e3699b9a858e70e006e270158ef14f4738ceb47,0x42b731c272a13e12b825363f6e58930ebb98d73c,0,transfer,7.761380e+04,0.000000e+00,7.761380e+04,7.761380e+04,0.032662,0.000000,0.057014,0.057014
257467,Ethereum,0xfa78354ac5d954b22741e422e4958da42ea28b2f,0x29ab34f5f609424d5f5e5af4ef6289e91f374bc0,0,transfer,2.249569e+06,0.000000e+00,2.249569e+06,2.249569e+06,0.497388,0.000000,1.000000,1.000000


In [167]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net'\
                ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share'
                ,'address'\
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [168]:
df_dollar = df_merge[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [169]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})

df_action_by_app = df_addr.groupby(['chain','project','address']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
})

df_action_by_app.reset_index(inplace=True)

for val in val_cols:
        df_action_by_app[val + '_pct_share'] = df_action_by_app.groupby(['chain','address'])[val].transform(lambda x: x / x.sum())

df_action_by_app = df_action_by_app.groupby(['chain','project']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})


df_action.reset_index(inplace=True)
df_action_by_app.reset_index(inplace=True)

df_action.name = 'df_action'
df_action_by_app.name = 'df_action_by_app'

# display(df_action[df_action['project'] == 'gmx'])
# display(df_action_by_app[df_action_by_app['project'] == 'gmx'])

In [170]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        d.fillna(0)
        # d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

,index,chain,project,contract,method,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,usd_value,usd_value_in,usd_value_out,usd_value_net,address,action
112238,112238,Ethereum,0xd0c1a75972defcc3c01a7d83caef1333935ea449,0,transfer,4.571467e-03,1.045129e-02,0.000000,0.000000,2.330217e+04,23302.172988,0.000000e+00,0.000000e+00,1,0xd0c1a75972defcc3c01a7d83caef1333935ea449 - 0...
54974,54974,Ethereum,0x636c1432fd5967cfd2486779e70053af7c5127bd,0,transfer,2.747508e-03,0.000000e+00,0.008520,0.008562,1.309207e+04,0.000000,1.309207e+04,1.309207e+04,1,0x636c1432fd5967cfd2486779e70053af7c5127bd - 0...
44999,44999,Ethereum,0x5092187cb7f221219d489a1fea228e4c8e580e4e,0,transfer,4.235302e-05,2.554856e-05,0.000059,0.000064,1.956682e+04,9783.407518,9.783408e+03,9.783408e+03,2,0x5092187cb7f221219d489a1fea228e4c8e580e4e - 0...
10624,10624,Ethereum,0x0eafc8ebaddb1a0ffed1aef70e451f47bea96b11,0,transfer,1.217043e-02,2.629589e-02,0.001883,0.000000,1.682779e+05,153212.753705,1.506516e+04,0.000000e+00,1,0x0eafc8ebaddb1a0ffed1aef70e451f47bea96b11 - 0...
59079,59079,Ethereum,0x6b1dd119a40eee47bf4031626d8ff0e4cd5577f3,0,0x,1.018191e-05,2.198676e-05,0.000000,0.000000,2.371458e+04,23714.578903,0.000000e+00,0.000000e+00,1,0x6b1dd119a40eee47bf4031626d8ff0e4cd5577f3 - 0...
57159,57159,Ethereum,0x6792a2d3cc38f8a66d23c07eb35963b165d2c6e8,0,transfer,3.645565e-03,0.000000e+00,0.010685,0.010685,1.374289e+04,0.000000,1.374289e+04,1.374289e+04,1,0x6792a2d3cc38f8a66d23c07eb35963b165d2c6e8 - 0...
75233,75233,Ethereum,0x8a68aa85ee7e8861eebd1dc516cdfe9f07046518,0,transfer,9.678293e-06,1.931899e-05,0.000000,0.000000,2.008733e+04,20087.331955,0.000000e+00,0.000000e+00,1,0x8a68aa85ee7e8861eebd1dc516cdfe9f07046518 - 0...
130975,130975,Ethereum,0xf4e8b379782ee6a4327b55729aa5ffc25f8664d5,0,transfer,2.823746e-02,5.812893e-02,0.000000,0.000000,7.206089e+04,72060.887485,0.000000e+00,0.000000e+00,1,0xf4e8b379782ee6a4327b55729aa5ffc25f8664d5 - 0...
137151,137151,Ethereum,curvefi,factory_metapool_tusdfraxbp,add_liquidity,6.980413e-02,0.000000e+00,0.140027,0.194117,1.449411e+06,0.000000,1.449411e+06,1.449411e+06,1,curvefi - factory_metapool_tusdfraxbp | add_li...
42990,42990,Ethereum,0x4cbd29359fe4da6182f8c3b9206587f66874d025,0,transfer,8.917767e-03,0.000000e+00,0.017274,0.017274,1.902176e+04,0.000000,1.902176e+04,1.902176e+04,1,0x4cbd29359fe4da6182f8c3b9206587f66874d025 - 0...


In [171]:
final_cols = ['chain','project','action','usd_value_out','usd_value_out_pct_share','usd_value_net','usd_value_net_pct_share','address','style','granularity']

In [172]:
style = 'Dollar-Weighted'
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

dfs = []
for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['granularity'] = 'action'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)

        print('Sum by App')
        sumdf = []
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['action'] = sumdf['project']
        sumdf['granularity'] = 'project'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)
        # print(sumdf[['usd_value_out_pct_share_100']].sum())

Dollar-Weighted
Sum by Action
Sum by App
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only

Sum by App
Sum by Action
Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1905355591.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only

In [173]:
style = 'Address-Weighted'
min_addresses = 3
print(style)
# print(df_action.columns)
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # display(tdf)
        tdf = tdf[tdf['address'] >= min_addresses]
        sumdf = tdf
        print('Sum by Action')
        # sumdf = tdf.groupby(['chain','project','action']).sum()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        # print(sumdf.columns)
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['granularity'] = 'action'
        sumdf = sumdf[final_cols]
        # display( sumdf.head(10) )
        # display(sumdf.head())
        dfs.append(sumdf)

        print('Sum by App')
        sumdf = df_action_by_app[df_action_by_app['chain'] == c]
        sumdf = sumdf[sumdf['address'] >= min_addresses]
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = sumdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share', ascending = False)
        sumdf['style'] = style
        sumdf['action'] = sumdf['project']
        sumdf['granularity'] = 'project'
        sumdf = sumdf[final_cols]

        # display(sumdf.head())
        dfs.append(sumdf)
        # display( sumdf.head(10) )
        # print(sumdf[['usd_value_out_pct_share_100']].sum()) #SHould be > 100 since it's avg by address

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1323552334.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1323552334.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


Address-Weighted
Sum by Action
Sum by App
Sum by Action
Sum by App
Sum by Action
Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_48870/1323552334.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


In [174]:
# Get uniques by chain
df_uniques = df_merge.groupby(['chain'])['address'].nunique()
df_uniques = df_uniques.reset_index()
df_uniques=df_uniques.rename(columns={'address':'total_addresses'})
df_uniques.head()

,chain,total_addresses
0,Arbitrum,582
1,Ethereum,21101
2,Optimism,233


In [178]:
pd_export = pd.concat(dfs)
pd_export = pd_export.merge(df_uniques, on='chain',how='left')
pd_export = pd_export[pd_export['usd_value_out_pct_share'] > (0.5 / 100) ] # greater than 0.5 %
pd_export['pct_addresses'] = pd_export['address'] / pd_export['total_addresses']

# display(pd_export[(pd_export['granularity'] == 'action') & (pd_export['project'] == 'Curve') & (pd_export['style'] == 'Address-Weighted') ])
pd_export.to_csv('csv_outputs/whale_agg_summary.csv')
display(pd_export)


,chain,project,action,usd_value_out,usd_value_out_pct_share,usd_value_net,usd_value_net_pct_share,address,style,granularity,total_addresses,pct_addresses
0,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,2.324114e+08,0.095237,1.974535e+06,0.001249,4.0,Dollar-Weighted,action,582,0.006873
1,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,1.391553e+08,0.057023,1.805717e+07,0.011419,1.0,Dollar-Weighted,action,582,0.001718
2,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,1.343188e+08,0.055041,1.343188e+08,0.084941,14.0,Dollar-Weighted,action,582,0.024055
3,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,1.206658e+08,0.049446,0.000000e+00,0.000000,1.0,Dollar-Weighted,action,582,0.001718
4,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,9.757784e+07,0.039985,9.757784e+07,0.061707,36.0,Dollar-Weighted,action,582,0.061856
...,...,...,...,...,...,...,...,...,...,...,...,...
288892,Optimism,Synapse,Synapse,3.544742e+05,0.074213,3.544742e+05,0.079664,6.0,Address-Weighted,project,233,0.025751
288893,Optimism,Kwenta,Kwenta,1.100147e+05,0.051978,1.100147e+05,0.058464,5.0,Address-Weighted,project,233,0.021459
288894,Optimism,OTHER,OTHER,1.237166e+05,0.037957,9.506361e+03,0.026540,188.0,Address-Weighted,project,233,0.806867
288895,Optimism,0xbe13e651ff246ac63fa36cad6cd8610b090a6a25,0xbe13e651ff246ac63fa36cad6cd8610b090a6a25,1.576269e+05,0.030869,1.124067e+03,0.000509,10.0,Address-Weighted,project,233,0.042918
